In [48]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime 

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

#Yahoo!Finance 模組
import yfinance as yf 

import warnings
warnings.filterwarnings('ignore')

In [49]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [50]:
print(tf.__version__)

2.12.0


In [51]:
tf.random.set_seed(42)
np.random.seed(42)

In [52]:
# https://finance.yahoo.com/most-active
# https://finance.yahoo.com/crypto/
symbol = "TSLA"
# Read data
df = yf.download(symbol,period='2y')
df_Indicators= yf.download(symbol,period='2y')
df.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-06-24,224.996674,232.539993,222.536667,226.606674,226.606674,137947200
2021-06-25,229.860001,231.270004,222.899994,223.956665,223.956665,97490100
2021-06-28,223.880005,231.566666,223.440002,229.573334,229.573334,64884600
2021-06-29,228.216660,229.169998,225.296661,226.919998,226.919998,52143900
2021-06-30,226.589996,230.936661,226.046661,226.566666,226.566666,56774700


In [53]:
df_Indicators.shape

(503, 6)

In [54]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
2021-08-03,239.666672,240.883331,233.669998,236.580002,236.580002,64860900,225.397145,221.019287,228.070692,224.360853,70.936113,1.000000,234.684094,206.810575,1.734263
2021-08-04,237.000000,241.633331,236.309998,236.973328,236.973328,51007800,227.935240,222.455477,230.296881,226.069454,72.676693,1.000000,237.255907,206.447761,2.559914
2021-08-05,238.666672,240.316666,237.136673,238.210007,238.210007,38758800,231.261431,224.131906,232.275516,227.710618,77.012204,1.000000,239.774485,205.989851,3.276271
2021-08-06,237.300003,238.776672,232.543335,233.033340,233.033340,46869000,233.743336,225.390954,232.464998,228.428819,68.378990,0.777712,240.956088,206.213248,3.387229
2021-08-09,236.723328,239.676666,235.043335,237.919998,237.919998,44145900,235.477145,226.659049,233.828885,229.707432,68.462283,0.779857,242.461124,205.643546,3.825380


In [55]:
df_Indicators.shape

(476, 15)

In [56]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_Indicators)
df_Indicators[df_Indicators.columns] = scaler.transform(df_Indicators[df_Indicators.columns])
df_Indicators.to_csv('TSLA-2y.csv')

In [9]:
n = len(df)
train_df = df_Indicators[0:int(n*0.7)]
val_df = df_Indicators[int(n*0.7):int(n*0.9)]
test_df = df_Indicators[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((352, 15), (100, 15), (24, 15))

In [10]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [11]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')